In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from preprocessing import traer_datasets
from preprocessing import separar_dataset
from preprocessing import feature_engineering_general

In [ ]:
from graficos_modelos import mostrar_reporte_clasificacion, graficar_auc_roc

In [ ]:
from funciones_auxiliares import encontrar_hiperparametros_RGSCV

### Obtención de los datos y preprocesamiento

Traemos datasets y particionamos

In [ ]:
df, df_sin_target, solo_target = traer_datasets()

X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)
X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_train.is_copy=False

y_train.set_index('id', inplace=True)
y_train.sort_values(by=['id'], inplace=True, ascending=True)

y_test.set_index('id', inplace=True)
y_test.sort_values(by=['id'], inplace=True, ascending=True)

Aplicamos metodo de feature engineering general

In [ ]:
X_train, X_test = feature_engineering_general(X_train, X_test)

### Búsqueda de Hiperparámetros

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento.

In [ ]:
params = {'criterion': ['gini','entropy'], 'min_samples_leaf':np.arange(1,16),'max_depth': np.arange(1,30)}
hiperparametros = encontrar_hiperparametros_RGSCV(DecisionTreeClassifier(), params=params, X=X_train, y=y_train)

In [ ]:
criterio_elegido = hiperparametros['criterion']
profundidad_elegida = hiperparametros['max_depth']
min_leaf_elegido = hiperparametros['min_samples_leaf']
print(f'Mejor criterio: {criterio_elegido}')
print(f'Mejor profundidad máxima: {profundidad_elegida}')
print(f'Mejor mínima cantidad de instancias por hoja: {min_leaf_elegido}')

### Entrenando el Modelo con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
    clf = DecisionTreeClassifier(max_depth=profundidad_elegida, min_samples_leaf=min_leaf_elegido, criterion=criterio_elegido, random_state=117)
    clf.fit(X_train.iloc[train_index], y_train.iloc[train_index])
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], clf.predict(X_train.iloc[test_index])))

### Predicción del modelo sobre holdout

Una vez entrenado y validado el modelo con CrossValidation, viendo que las métricas resultantes fueron razonables y no dan indicio de overfit o mala división del dataset, pasaremos a probar el modelo en la partición Holdout. Para esto usaremos los mejores hiperparámetros hallados previamente en Training.

In [ ]:
arbol_holdout = DecisionTreeClassifier(max_depth=profundidad_elegida, min_samples_leaf=min_leaf_elegido, criterion=criterio_elegido, random_state=117)
arbol_holdout.fit(X_train.to_numpy(), y_train.to_numpy())

In [ ]:
y_pred = arbol_holdout.predict(X_test.to_numpy())
y_pred_proba = arbol_holdout.predict_proba(X_test.to_numpy())[:, 1]
print(classification_report(y_test['llovieron_hamburguesas_al_dia_siguiente'].to_numpy(), y_pred))

In [ ]:
graficar_auc_roc(y_test['llovieron_hamburguesas_al_dia_siguiente'], y_pred_proba)